# LDA & Word2Vec with hyperparameter training

Hi everyone,
In this kernel I'd like to specifically talk to you about two machine learning techniques used for text analysis: Word2Vec and LDA. My main goal is to show you how you could approach a problem in general and how to do hyperparameter training.

In every data science task, I see four main steps in the process of solving a problem. These will by the way also be my main chapters:
1. <a href="#feature_inspection">Feature inspection</a>
2. <a href="#feature_creation">Feature creation</a>
3. <a href="#model_training">Model training</a>
4. <a href="#model_selection">Model selection</a>

Personally, I think you should spend most time in chapters 1 and 2, since I believe your model is as good as the data you input.

In section 1 we will inspect some basic statistics of the dataset.<br/>
In section 2 the preprocessing of the data and the LDA and Word2Vec model will be explained and applied.<br/>
In section 3 we will see how a grid search can be applied for hyperparameter training.<br/>
In section 4 we will select the model and make a submission.

If you read this kernel and are eager to give me some feedback, in particular I'm interested in performance improvements and writing neat code. But of course, any feedback is welcome. So please let me know when you have remarks! :)

First of all, let's import the data and specify all imports we will use in this kernel.

In [ ]:
# more common imports
import pandas as pd
import numpy as np
from collections import Counter
import re

# languange processing imports
import nltk
from gensim.corpora import Dictionary
# preprocessing imports
from sklearn.preprocessing import LabelEncoder

# model imports
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.word2vec import Word2Vec
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
# hyperparameter training imports
from sklearn.model_selection import GridSearchCV

# visualization imports
from IPython.display import display
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import base64
import io
%matplotlib inline
sns.set()  # defines the style of the plots to be seaborn style

In [ ]:
train_data = pd.read_csv('../input/train.csv')

print(train_data.shape)
train_data.head(3)

# 1) Feature Inspection <a id="feature_inspection"></a>

In order to get some feeling for the data we are going to work with, we will start out with a quick exploratory phase. We will have a look at a couple of things. For example how many documents (I will define one entry in the text column as a document) we have per writer. Look at the lengths of the documents. Look at some (in)frequent words that are used.

This exploratory phase is always a good thing to start with to get an idea of the data and it will probably help you out making decisions later on in the process of any data science problem you have and in particular now for text classification.

Just for reference, the writers we are talking about are EAP: Edgar Allan Poe, HPL: HP Lovecraft; MWS: Mary Wollstonecraft Shelley.

In [ ]:
# check if there's missing data
train_data.isnull().sum()

## Inspect author variable

In [ ]:
# check if we indeed have three authors
train_data.author.value_counts().index

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(8,6))

author_vc = train_data.author.value_counts()

ax.bar(range(3), author_vc)
ax.set_xticks(range(3))
ax.set_xticklabels(author_vc.index, fontsize=16)

for rect, c, value in zip(ax.patches, ['b', 'r', 'g'], author_vc.values):
    rect.set_color(c)
    height = rect.get_height()
    width = rect.get_width()
    x_loc = rect.get_x()
    ax.text(x_loc + width/2, 0.9*height, value, ha='center', va='center', fontsize=18, color='white')

It's good to see that the sizes of the three classes aren't too far apart. This means that the models shouldn't become too biased to one writer.

## Inspect text variable

In [ ]:
document_lengths = np.array(list(map(len, train_data.text.str.split(' '))))

print("The average number of words in a document is: {}.".format(np.mean(document_lengths)))
print("The minimum number of words in a document is: {}.".format(min(document_lengths)))
print("The maximum number of words in a document is: {}.".format(max(document_lengths)))

Note how there's actually a document with just two words. That is going to be interesting to classify. Let's dig in a little bit deeper and see the distribution of the number of words in a document.

In [ ]:
fig, ax = plt.subplots(figsize=(15,6))

ax.set_title("Distribution of number of words", fontsize=16)
ax.set_xlabel("Number of words")
sns.distplot(document_lengths, bins=50, ax=ax);

So there's very few document with over 150 words. Still, I would like to see the number of smaller documents a bit better. So we'll filter out the longer document lengths for the plot.

In [ ]:
print("There are {} documents with over 150 words.".format(sum(document_lengths > 150)))

shorter_documents = document_lengths[document_lengths <= 150]

In [ ]:
fig, ax = plt.subplots(figsize=(15,6))

ax.set_title("Distribution of number of words", fontsize=16)
ax.set_xlabel("Number of words")
sns.distplot(shorter_documents, bins=50, ax=ax);

This makes it a bit clearer. The shorter documents will probably be harder to classify since we'll have less words to cling to. LDA for example tries to find topics in documents, but if the documents are so short, perhaps it will find it hard to really find a topic in a two-word document.<br/>
Apparently there's actually quite some short documents. Let's just have a quick peek at some of these documents.

In [ ]:
print("There are {} documents with tops 5 words.".format(sum(document_lengths <= 5)))

In [ ]:
train_data[document_lengths == 2]

In [ ]:
train_data[document_lengths == 3]

Some interesting sentences. I was afraid there'd be some random sentences like 'This is it.' or 'So scary!', but I think that is not really the case here. One fun fact here: HP Lovecraft has no documents of 2 or 3 words ;) At least for the training data..

# 2) Feature creation <a id="feature_creation"></a>

The usual approach in natural language processing, is to first kind of cleanse the text. We have to make sure our model can understand similarities and understand when two different words mean similar things. You can't just input the raw text into a model and expect it to understand everything. Therefor, I summarized the most common steps to take in order to achieve this cleansing.

1. Tokenize the text; split sentences in (good) words. Thus watch out for punctuation.
2. Remove words that mean little; these are usually words that occur very frequently or words that occur very infrequently. Also, punctuation can be removed, but perhaps you'd like to keep some punctuation in, like exclamation marks or question marks since maybe one writer uses them more than others.
3. Stemming; this basically means joining words that mean the same. Take for example the words running and runs, the stem of both words is run. Thus with stemming you'd group these words together and give them the same meaning for the model.
4. Vectorize words; since we can't input plain words into a model and expect it to learn from it, we have to vectorize the words. This basically means create unit vectors for all words.

For LDA you'd like to perform all four steps. However, for w2v you'd only want to tokenize and remove some punctuation. The w2v model can determine by itself what words are important and what are not, but we'll get back to that later on.


## 2.1) Tokenize the text

I noticed there's some non-ascii characters present. I want to simply filter them out. For LDA I want to completely remove these words since it's highly likely the words with invalid characters appear only once and therefor don't have much value for the topics anyway. For word2vec I want to keep the order and the number of words per sentence the same; therefor I will replace these words with a random word 'qwerty'. Why the number of words and the order is important, will be clear later on when we discuss the model. Also, the input for the model will be sentences and that's why I will use '.' as the delimiter for sentences.

In [ ]:
# find and remove non-ascii words
# I stored our special word in a variable for later use
our_special_word = 'qwerty'

def remove_ascii_words(df):
    """ removes non-ascii characters from the 'texts' column in df.
    It returns the words containig non-ascii characers.
    """
    non_ascii_words = []
    for i in range(len(df)):
        for word in df.loc[i, 'text'].split(' '):
            if any([ord(character) >= 128 for character in word]):
                non_ascii_words.append(word)
                df.loc[i, 'text'] = df.loc[i, 'text'].replace(word, our_special_word)
    return non_ascii_words

non_ascii_words = remove_ascii_words(train_data)

print("Replaced {} words with characters with an ordinal >= 128 in the train data.".format(
    len(non_ascii_words)))

In [ ]:
def get_good_tokens(sentence):
    replaced_punctation = list(map(lambda token: re.sub('[^0-9A-Za-z!?]+', '', token), sentence))
    removed_punctation = list(filter(lambda token: token, replaced_punctation))
    return removed_punctation

A little codeblock specifically meant for word2vec.

In [ ]:
# Here we get transform the documents into sentences for the word2vecmodel
# we made a function such that later on when we make the submission, we don't need to write duplicate code
def w2v_preprocessing(df):
    """ All the preprocessing steps for word2vec are done in this function.
    All mutations are done on the dataframe itself. So this function returns
    nothing.
    """
    df['text'] = df.text.str.lower()
    df['document_sentences'] = df.text.str.split('.')  # split texts into individual sentences
    df['tokenized_sentences'] = list(map(lambda sentences:
                                         list(map(nltk.word_tokenize, sentences)),
                                         df.document_sentences))  # tokenize sentences
    df['tokenized_sentences'] = list(map(lambda sentences:
                                         list(map(get_good_tokens, sentences)),
                                         df.tokenized_sentences))  # remove unwanted characters
    df['tokenized_sentences'] = list(map(lambda sentences:
                                         list(filter(lambda lst: lst, sentences)),
                                         df.tokenized_sentences))  # remove empty lists

w2v_preprocessing(train_data)

So we're actually already finished the word2vec preprocessing! All the following steps are preprocessing steps for LDA.

In [ ]:
def lda_get_good_tokens(df):
    df['text'] = df.text.str.lower()
    df['tokenized_text'] = list(map(nltk.word_tokenize, df.text))
    df['tokenized_text'] = list(map(get_good_tokens, df.tokenized_text))

lda_get_good_tokens(train_data)

Let's view at the top word frequencies now and get an idea of the words that are used mostly.

In [ ]:
tokenized_only_dict = Counter(np.concatenate(train_data.tokenized_text.values))

tokenized_only_df = pd.DataFrame.from_dict(tokenized_only_dict, orient='index')
tokenized_only_df.rename(columns={0: 'count'}, inplace=True)

In [ ]:
tokenized_only_df.sort_values('count', ascending=False, inplace=True)

In [ ]:
# I made a function out of this since I will use it again later on 
def word_frequency_barplot(df, nr_top_words=50):
    """ df should have a column named count.
    """
    fig, ax = plt.subplots(1,1,figsize=(20,5))

    sns.barplot(list(range(nr_top_words)), df['count'].values[:nr_top_words], palette='hls', ax=ax)

    ax.set_xticks(list(range(nr_top_words)))
    ax.set_xticklabels(df.index[:nr_top_words], fontsize=14, rotation=90)
    return ax
    
ax = word_frequency_barplot(tokenized_only_df)
ax.set_title("Word Frequencies", fontsize=16);

It was to be expected that simple words like 'the', 'of' and 'to' are words that are used often. Unless you think that one author would preferably use on of these words more often than other authors, it's safe to remove these words. Like we stated in point 2 above, we will remove frequent words and stopwords since they probably bring little meaning and maybe even create noise when we want to classify later on.

## 2.2) Remove words that mean little

As stated above, words the have little meaning to the model are words we'd like to remove. Moreover, these words can create noise which can make it harder to classify the texts later on.

In [ ]:
def remove_stopwords(df):
    """ Removes stopwords based on a known set of stopwords
    available in the nltk package. In addition, we include our
    made up word in here.
    """
    # Luckily nltk already has a set of stopwords that we can remove from the texts.
    stopwords = nltk.corpus.stopwords.words('english')
    # we'll add our own special word in here 'qwerty'
    stopwords.append(our_special_word)

    df['stopwords_removed'] = list(map(lambda doc:
                                       [word for word in doc if word not in stopwords],
                                       df['tokenized_text']))

remove_stopwords(train_data)

## 2.3) Stemming

For the stemming part, we'll start with lemmatization, this will stem words like leaves to leaf. Afterwards, we'll apply the porterstemmer which in turn will complete the stemming by converting words like running to run.

In [ ]:
def stem_words(df):
    lemm = nltk.stem.WordNetLemmatizer()
    df['lemmatized_text'] = list(map(lambda sentence:
                                     list(map(lemm.lemmatize, sentence)),
                                     df.stopwords_removed))

    p_stemmer = nltk.stem.porter.PorterStemmer()
    df['stemmed_text'] = list(map(lambda sentence:
                                  list(map(p_stemmer.stem, sentence)),
                                  df.lemmatized_text))

stem_words(train_data)

## 2.4) Vectorize words

Here we create a dictionary for the model. Since a computer can't reallly understand what a word means, we need to transform it into some number the model will be able to work with. We use gensim.corpora.Dictionary for this.

In [ ]:
dictionary = Dictionary(documents=train_data.stemmed_text.values)

print("Found {} words.".format(len(dictionary.values())))

In [ ]:
dictionary.filter_extremes(no_above=0.8, no_below=3)

dictionary.compactify()  # Reindexes the remaining words after filtering
print("Left with {} words.".format(len(dictionary.values())))

I decided to remove only the top 20% of words since we're dealing with a relatively small dataset here. Also words that occur less than 3 times are removed.



In [ ]:
#Make a BOW for every document
def document_to_bow(df):
    df['bow'] = list(map(lambda doc: dictionary.doc2bow(doc), df.stemmed_text))
    
document_to_bow(train_data)

In [ ]:
# we make a function such that later on when we make the submission, we don't need to write duplicate code
def lda_preprocessing(df):
    """ All the preprocessing steps for LDA are combined in this function.
    All mutations are done on the dataframe itself. So this function returns
    nothing.
    """
    lda_get_good_tokens(df)
    remove_stopwords(df)
    stem_words(df)
    document_to_bow(df)

## Visualize the cleansed words frequencies

Excellent! We cleansed our data and are ready to continue with LDA and Word2Vec, but before we do that, I would like to visualize the cleansed results and see what we created so far.

In [ ]:
cleansed_words_df = pd.DataFrame.from_dict(dictionary.token2id, orient='index')
cleansed_words_df.rename(columns={0: 'id'}, inplace=True)

cleansed_words_df['count'] = list(map(lambda id_: dictionary.dfs.get(id_), cleansed_words_df.id))
del cleansed_words_df['id']

In [ ]:
cleansed_words_df.sort_values('count', ascending=False, inplace=True)

In [ ]:
ax = word_frequency_barplot(cleansed_words_df)
ax.set_title("Document Frequencies (Number of documents a word appears in)", fontsize=16);

Looks a bit more interesting right now that we have filtered out those common words ;) It's also funny how we ended up seeing our question mark in this plot.<br/>
Something I wanted to try out is see how often each of the writers used these words in the plot above and see if we can already find something interesting.

In [ ]:
EAP_words = list(np.concatenate(train_data.loc[train_data.author == 'EAP', 'stemmed_text'].values))
HPL_words = list(np.concatenate(train_data.loc[train_data.author == 'HPL', 'stemmed_text'].values))
MWS_words = list(np.concatenate(train_data.loc[train_data.author == 'MWS', 'stemmed_text'].values))

In [ ]:
EAP_word_frequencies = {word: EAP_words.count(word) for word in cleansed_words_df.index[:50]}
HPL_word_frequencies = {word: HPL_words.count(word) for word in cleansed_words_df.index[:50]}
MWS_word_frequencies = {word: MWS_words.count(word) for word in cleansed_words_df.index[:50]}

In [ ]:
frequencies_df = pd.DataFrame(index=cleansed_words_df.index[:50])

In [ ]:
frequencies_df['MWS_freq'] = list(map(lambda word:
                                      MWS_word_frequencies[word],
                                      frequencies_df.index))
frequencies_df['MWS_HPL_freq'] = list(map(lambda word:
                                          MWS_word_frequencies[word] + HPL_word_frequencies[word],
                                          frequencies_df.index))
frequencies_df['MWS_HPL_EAP_freq'] = list(map(lambda word:
                                              MWS_word_frequencies[word] + HPL_word_frequencies[word] + EAP_word_frequencies[word],
                                              frequencies_df.index))

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(20,5))

nr_top_words = len(frequencies_df)
nrs = list(range(nr_top_words))
sns.barplot(nrs, frequencies_df['MWS_HPL_EAP_freq'].values, color='b', ax=ax, label="EAP")
sns.barplot(nrs, frequencies_df['MWS_HPL_freq'].values, color='g', ax=ax, label="HPL")
sns.barplot(nrs, frequencies_df['MWS_freq'].values, color='r', ax=ax, label="MWS")

ax.set_title("Word frequencies per author", fontsize=16)
ax.legend(prop={'size': 16})
ax.set_xticks(nrs)
ax.set_xticklabels(frequencies_df.index, fontsize=14, rotation=90);

It's good to see that this plot suggests that our three writers prefer some words over others. Take for example word 'thing', which isn't used much by MWS compared to EAP and HPL. On the other hand, she clearly uses the word 'love' more than the other two writers.

Note: the difference between the last two barplots, is that the first barplot depicts document frequencies (so the number of times a word appears in a document) and the second plot shows how often each word is used (per author). Therefor the bars are not fully ordered by height and therefor the bars in the second plot are slightly higher.

# 3) Model Training <a id="model_training"></a>

## LDA

Latent Dirichlet Allocation (LDA) is generative approach in classifying texts. It is a three level hierarchical Bayesian model where it creates probabilities on word level, on document level and on corpus level (corpus means all documents). The objective function looks pretty complex, but it's just a way of describing the probability of a corpus:

$$p(\mathcal{D} | \alpha, \eta) = \prod_{m=1}^{M} \int p(\theta_{m} | \alpha) \prod_{n=1}^{N_{m}} \sum_{i=1}^{k} p(z_{mn}^{i} | \theta_{m}) p(w_{mn} | z_{mn}^{i}, \eta) d\theta_{m}$$

where $w_{mn}$ represents word $n$ from document $m$. The three level hierarchical Bayesian model can now be summarized as follows. First of all, parameters $\alpha$ and $\eta$ are corpus level parameters. Second, variables $\theta_{1},...,\theta_{M}$ are document level parameters. Third, all the $z_{mn}^{i}$ and $w_{mn}$ are word level variables. So we have $M$ documents, we have $k$ topics and we have $N_{m}$ words per document.

This is the objective function that we want to optimize with LDA; we want to maximize the probability of the corpus in the training set.

There's one more thing I'd like to point out, before we start training the model. We have to pick two priors: Dir($\alpha$) for the topic distribution and a Multinomial($\theta$) for the words (given a topic). Usually both priors are uniformly distributed, however, it is only natural to assume that some topics occur more often than others, therefor we will give $\alpha$ an asymmetric prior. We could do the same for $\theta$ for the word distributions, but that wouldn't make sense anymore for the following reason. We don't know what words in what topics appear with greater probabilities. For that reason, we keep the uniform distribution the prior distribution and let the model learn by itself what words in what topics appear with greater probabilities.

## LDA training


In [ ]:
corpus = train_data.bow

In [ ]:
%%time
num_topics = 150
#A multicore approach to decrease training time
LDAmodel = LdaMulticore(corpus=corpus,
                        id2word=dictionary,
                        num_topics=num_topics,
                        workers=4,
                        chunksize=4000,
                        passes=7,
                        alpha='asymmetric')

## LDA topic inspection

Let's first create the feature vectors.

In [ ]:
def document_to_lda_features(lda_model, document):
    """ Transforms a bag of words document to features.
    It returns the proportion of how much each topic was
    present in the document.
    """
    topic_importances = LDAmodel.get_document_topics(document, minimum_probability=0)
    topic_importances = np.array(topic_importances)
    return topic_importances[:,1]

train_data['lda_features'] = list(map(lambda doc:
                                      document_to_lda_features(LDAmodel, doc),
                                      train_data.bow))

Now let's continue with the author individual topic distributions and let's see some words that come with the topics.

In [ ]:
EAP_topic_distribution = train_data.loc[train_data.author == 'EAP', 'lda_features'].mean()
HPL_topic_distribution = train_data.loc[train_data.author == 'HPL', 'lda_features'].mean()
MWS_topic_distribution = train_data.loc[train_data.author == 'MWS', 'lda_features'].mean()

In [ ]:
fig, [ax1,ax2,ax3] = plt.subplots(3,1,figsize=(20,10))

nr_top_bars = 5

ax1.set_title("EAP topic distributions", fontsize=16)
ax2.set_title("HPL topic distributions", fontsize=16)
ax3.set_title("MWS topic distributions", fontsize=16)
ax3.text(-10, 0.04, "Average Probability of Topic", fontsize=30, ha="center", va="center",
         rotation="vertical")

for ax, distribution, color in zip([ax1,ax2,ax3],
                                   [EAP_topic_distribution,HPL_topic_distribution,MWS_topic_distribution],
                                   ['b','g','r']):
    # Individual distribution barplots
    ax.bar(range(len(distribution)), distribution, alpha=0.7)
    rects = ax.patches
    for i in np.argsort(distribution)[-nr_top_bars:]:
        rects[i].set_color(color)
        rects[i].set_alpha(1)
    # General plotting adjustments
    ax.set_xlim(-1, 150)
    ax.set_xticks(range(20,149,20))
    ax.set_xticklabels(range(20,149,20), fontsize=16)
    ax.set_ylim(0,0.02)
    ax.set_yticks([0,0.01,0.02])
    ax.set_yticklabels([0,0.01,0.02], fontsize=16)

fig.tight_layout(h_pad=3.)

For the sake of classification, it's good to see that the top topics differ per author :) Let's have a look at what words these top topics contain.

In [ ]:
def get_topic_top_words(lda_model, topic_id, nr_top_words=5):
    """ Returns the top words for topic_id from lda_model.
    """
    id_tuples = lda_model.get_topic_terms(topic_id, topn=nr_top_words)
    word_ids = np.array(id_tuples)[:,0]
    words = map(lambda id_: lda_model.id2word[id_], word_ids)
    return words

In [ ]:
for author, distribution in zip(['EAP', 'HPL', 'MWS'], [EAP_topic_distribution, HPL_topic_distribution, MWS_topic_distribution]):
    print("Looking up top words from top topics from {}.".format(author))
    for x in sorted(np.argsort(distribution)[-5:]):
        top_words = get_topic_top_words(LDAmodel, x)
        print("For topic {}, the top words are: {}.".format(x, ", ".join(top_words)))
    print("")

In [ ]:
def show_image(base64_encoded_image):
    """ Decodes a base64 encoded image and plots it.
    """
    fig, ax = plt.subplots(figsize=(10,10))

    decoded_image = base64.b64decode(base64_encoded_image)
    img = io.BytesIO(decoded_image)
    img = mpimg.imread(img, format='PNG')

    ax.imshow(img)
    ax.axis('off');

## Word2Vec

Word2vec is a deep learning method for creating vector representations of words. There are basically two word2vec approaches: continuous skip-gram modeling (SG) or continuous bag of words (CBOW). Both create word vector representations, but with different approaches. The basic idea for skip-gram modeling, is given a word $w_{i}$, predict the context $w_{i-2}$, $w_{i-1}$, $w_{i+1}$, $w_{i+2}$ of this word. Whereas for the continuous bag of words approach, this process is reversed; given the context $w_{i-2}$, $w_{i-1}$, $w_{i+1}$, $w_{i+2}$, predict word $w_{i}$.

The network itself is very shallow as you can see from the image.

<img src="w2v_layers.png" style="width: 40%;">

In [ ]:
w2v_layers_png = b'...'
show_image(w2v_layers_png)

Where $V$ is vocabulary size and $N$ is hidden layer size. This is the network in it's simplest form, since here there's only one input vector (basically you'd have given word $w_{i}$ predict word $w_{i+1}$). So the matrix we're eventually interested in, is matrix $A$ with where we have a linear activation function. The following image shows how the where we get the vector representations for the individual words come from.

In [ ]:
word_vector_length_png = b'...'
show_image(word_vector_length_png)

Thus, after training, if for example you'd like to have the vector representation of the $j^{th}$ word in the vocabulary, then you should get the $j^{th}$ row of matrix $A$ (you're basically multiplying the $j^{th}$ unit vector with matrix $A$ and you're left with the vector representation).

## Word2Vec training

In [ ]:
sentences = []
for sentence_group in train_data.tokenized_sentences:
    sentences.extend(sentence_group)

print("Number of sentences: {}.".format(len(sentences)))
print("Number of texts: {}.".format(len(train_data)))

According to our preprocessig steps for word2vec, we got about a 1000 more sentences than texts. Which means the majority of texts are simply one sentence. Let's train the word2vec model.

In [ ]:
%%time
# Set values for various parameters
num_features = 200    # Word vector dimensionality
min_word_count = 3    # Minimum word count
num_workers = 4       # Number of threads to run in parallel
context = 6           # Context window size
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model
W2Vmodel = Word2Vec(sentences=sentences,
                    sg=1,
                    hs=0,
                    workers=num_workers,
                    size=num_features,
                    min_count=min_word_count,
                    window=context,
                    sample=downsampling,
                    negative=5,
                    iter=6)

## Word2Vec feature inspection

Let's first create the word2vec features.

In [ ]:
def get_w2v_features(w2v_model, sentence_group):
    """ Transform a sentence_group (containing multiple lists
    of words) into a feature vector. It averages out all the
    word vectors of the sentence_group.
    """
    words = np.concatenate(sentence_group)  # words in text
    index2word_set = set(w2v_model.wv.vocab.keys())  # words known to model
    
    featureVec = np.zeros(w2v_model.vector_size, dtype="float32")
    
    # Initialize a counter for number of words in a review
    nwords = 0
    # Loop over each word in the comment and, if it is in the model's vocabulary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            featureVec = np.add(featureVec, w2v_model[word])
            nwords += 1.

    # Divide the result by the number of words to get the average
    if nwords > 0:
        featureVec = np.divide(featureVec, nwords)
    return featureVec

train_data['w2v_features'] = list(map(lambda sen_group:
                                      get_w2v_features(W2Vmodel, sen_group),
                                      train_data.tokenized_sentences))

In [ ]:
EAP_w2v_distribution = train_data.loc[train_data.author == 'EAP', 'w2v_features'].mean()
HPL_w2v_distribution = train_data.loc[train_data.author == 'HPL', 'w2v_features'].mean()
MWS_w2v_distribution = train_data.loc[train_data.author == 'MWS', 'w2v_features'].mean()

In [ ]:
fig, [ax1,ax2,ax3] = plt.subplots(3,1,figsize=(20,10))

nr_top_bars = 5

ax1.set_title("EAP w2v feature distributions", fontsize=16)
ax2.set_title("HPL w2v feature distributions", fontsize=16)
ax3.set_title("MWS w2v feature distributions", fontsize=16)
ax3.text(-10, 2.3, "Average feature vectors", fontsize=30, ha="center", va="center", rotation="vertical")

for ax, distribution, color in zip([ax1,ax2,ax3], [EAP_w2v_distribution,HPL_w2v_distribution,MWS_w2v_distribution], ['b','g','r']):
    # Individual distribution barplots
    ax.bar(range(len(distribution)), distribution, alpha=0.7)
    rects = ax.patches
    for i in np.argsort(distribution)[-nr_top_bars:]:
        rects[i].set_color(color)
        rects[i].set_alpha(1)
    # General plotting adjustments
    ax.set_xlim(-1, 200)
    ax.set_xticks(range(20,199,20))
    ax.set_xticklabels(range(20,199,20), fontsize=16)
    ax.set_ylim(-0.8,0.8)

fig.tight_layout(h_pad=3.)

Clearly this doesn't look too promising haha, but we'll give it a go!

## Classification and hyperparameter tuning

We have come far now! At this point we transformed the documents we had into features we can use to train our final models. We used LDA and Word2Vec to extract sensible feature data from the documents. Now we're going to apply a few supervised classifiers to be able to predict what text belongs to which writer.

Before we start classifying, we have to transform our author labels (EAP, HPL and MWS) into numbers. We can do this with the sklearn LabelEncoder. After that we'll create a function with which we can easily train and test our models.

In [ ]:
label_encoder = LabelEncoder()

label_encoder.fit(train_data.author)
train_data['author_id'] = label_encoder.transform(train_data.author)

In [ ]:
def get_cross_validated_model(model, param_grid, X, y, nr_folds=5):
    """ Trains a model by doing a grid search combined with cross validation.
    args:
        model: your model
        param_grid: dict of parameter values for the grid search
    returns:
        Model trained on entire dataset with hyperparameters chosen from best results in the grid search.
    """
    # train the model (since the evaluation is based on the logloss, we'll use neg_log_loss here)
    grid_cv = GridSearchCV(model, param_grid=param_grid, scoring='neg_log_loss', cv=nr_folds, n_jobs=-1, verbose=True)
    best_model = grid_cv.fit(X, y)
    # show top models with parameter values
    result_df = pd.DataFrame(best_model.cv_results_)
    show_columns = ['mean_test_score', 'mean_train_score', 'rank_test_score']
    for col in result_df.columns:
        if col.startswith('param_'):
            show_columns.append(col)
    display(result_df[show_columns].sort_values(by='rank_test_score').head())
    return best_model

In [ ]:
# since train_data['lda_features'] and train_data['w2v_features'] don't have the needed shape and type yet,
# we first have to transform every entry
X_train_lda = np.array(list(map(np.array, train_data.lda_features)))
X_train_w2v = np.array(list(map(np.array, train_data.w2v_features)))
X_train_combined = np.append(X_train_lda, X_train_w2v, axis=1)

Note: since we are dealing with log loss and not simply accuracy, we have to choose models that support the method predict_proba. If you're interested in what classifiers in the sklearn package support this method, you can execute the next cell.

from sklearn.utils.testing import all_estimators

estimators = all_estimators()

for name, class_ in estimators:
    if hasattr(class_, 'predict_proba'):
        print(name)

In [ ]:
# store all models in a dictionary
models = dict()

## Logistic Regression

In [ ]:
# LDA features only
lr = LogisticRegression()

param_grid = {'penalty': ['l1', 'l2']}

best_lr_lda = get_cross_validated_model(lr, param_grid, X_train_lda, train_data.author_id)

models['best_lr_lda'] = best_lr_lda

In [ ]:
# Word2Vec features only
lr = LogisticRegression()

param_grid = {'penalty': ['l1', 'l2']}

best_lr_w2v = get_cross_validated_model(lr, param_grid, X_train_w2v, train_data.author_id)

models['best_lr_w2v'] = best_lr_w2v

In [ ]:
# LDA and Word2Vec combined
lr = LogisticRegression()

param_grid = {'penalty': ['l1', 'l2']}

best_lr_combined = get_cross_validated_model(lr, param_grid, X_train_combined, train_data.author_id)

models['best_lr_combined'] = best_lr_combined

First of all, I'm actually surprised the word2vec features perform so well! But more importantly, combining the two features gives us the best performance :D

# 4) Model Selection <a id="model_selection"></a>

Awesome! We've come to the last step. So now we've trained a coupe of models and stored them all in the 'models' variable. I think this is a great and very easy approach to finding the best model. I just sort of randomly logistic regression for this kernel, but we could have tried out other models as well!

Now let's get clear what model performed best.

In [ ]:
for name, model in models.items():
    print("Model {} has a test score of: {:0.4f}".format(name, float(model.best_score_)))

Thus logistic regression with combined LDA and word2vec features wins!

## Make a submission

In [ ]:
test_data = pd.read_csv('../input/test.csv')

print(test_data.shape)
test_data.head(3)

In [ ]:
non_ascii_words = remove_ascii_words(test_data)

print("Replaced {} words with characters with an ordinal >= 128 in the test data.".format(
    len(non_ascii_words)))

Let's repeat our preprocessing steps now. Luckily, we made a nice function that does everything for us now.

In [ ]:
lda_preprocessing(test_data)
w2v_preprocessing(test_data)

In [ ]:
test_data['lda_features'] = list(map(lambda doc:
                                     document_to_lda_features(LDAmodel, doc),
                                     test_data.bow))

test_data['w2v_features'] = list(map(lambda sen_group:
                                     get_w2v_features(W2Vmodel, sen_group),
                                     test_data.tokenized_sentences))

In [ ]:
train_data['author_id'] = label_encoder.transform(train_data.author)

In [ ]:
X_test_lda = np.array(list(map(np.array, test_data.lda_features)))
X_test_w2v = np.array(list(map(np.array, test_data.w2v_features)))
X_test_combined = np.append(X_test_lda, X_test_w2v, axis=1)

Now it's time to make the classifications.

In [ ]:
submission_predictions = best_lr_combined.predict_proba(X_test_combined)

In [ ]:
label_encoder.classes_  # exactly the order in which we need to submit the writers 

In [ ]:
result = np.append(test_data.id.values.reshape(-1,1), submission_predictions, axis=1)

In [ ]:
submission_df = pd.DataFrame(data=result, columns=['id', 'EAP', 'HPL', 'MWS'])

In [ ]:
submission_df.to_csv('submission.csv', index=False)